In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split

In [2]:
edges = np.loadtxt('Gowalla_edges.txt',dtype=int)

users = np.unique(edges)
n = len(users)
ed = len(edges)
connections = []
s_data = np.ones(ed, dtype=int)

for i in range(0, ed):
    connections.append((edges[i][0], edges[i][1]))
    
for i in range(0, n):
    connections.append((i, i))
    
print("Сработало")

Сработало


In [3]:
def affinity_propagation(s_data, connections, iterations_number, s_s, t_p):
    
    s_matrix = np.append(s_data, np.repeat(s_s, n))
    r_matrix = np.zeros(s_matrix.shape[0], dtype=int)
    a_matrix = np.copy(r_matrix)
    
    result = {}
    
    for j in range(0, iterations_number):
        
        start = time.time()
        
        print("Итерация #" + str(j+1))
        
        #ответственность
        
        sum_temp = s_matrix + a_matrix
        
        print(a_matrix)
        print(r_matrix)
        
        max_1 = np.zeros(n)
        max_1.fill(-np.inf)     
        max_2 = np.copy(max_1)
        
        max_p = np.zeros(n)
        max_p.fill(np.inf)

        for i, pair in enumerate(connections):
            
            if max_1[pair[0]] < sum_temp[i]:
                max_2[pair[0]] = max_1[pair[0]]
                max_1[pair[0]] = sum_temp[i]
                max_p[pair[0]] = pair[1]
            else:
                if max_2[pair[0]] < sum_temp[i]:
                    max_2[pair[0]] = sum_temp[i]
        
        new_r_matrix = s_matrix.copy()
        
        for i, pair in enumerate(connections):
            if max_p[pair[0]] == pair[1]:
                new_r_matrix[i] -= max_2[pair[0]]
            else:
                new_r_matrix[i] -= max_1[pair[0]]
                
        r_matrix = np.copy(new_r_matrix)
        
        sum_temp = []
                
        #доступность
        
        r_matrix_copy = r_matrix.copy()
        
        for i in range(0, ed):
            if r_matrix_copy[i] < 0:
                r_matrix_copy[i] = 0
        
        positive_r = np.repeat(0, n)
    
        for i, pair in enumerate(connections):
            positive_r[pair[1]] += r_matrix_copy[i]
        for i, pair in enumerate(connections):
            a_matrix[i] = positive_r[pair[1]] - r_matrix_copy[i]
        for i in range(0, len(edges)):
            a_matrix[i] = np.min([0, a_matrix[i]])
        
        stop = time.time()
        print(stop - start)
    
    #get cluster leaders

    temp = np.zeros(n)
    temp.fill(-np.inf)
    
    sum_temp = r_matrix + a_matrix

    for i, pair in enumerate(connections):
        if sum_temp[i] > temp[pair[0]]:
            temp[pair[0]] = sum_temp[i]
            result[pair[0]] = pair[1]
            
    for i in range(t_p):        
        for a in result.keys(): 
            b = result[a] 
            if result[b] != b: 
                с = result[b] 
                result[a] = с 
    
    return result

In [4]:
def get_tp(data, pred_data, k):
    try:
        data = set(data[:k])
    except:
        data = set(data)
    pred_data = set(pred_data)
    return len(data.intersection(pred_data))

In [5]:
iterations_number = 10
s_s = -2
t_p = 5

In [6]:
result = affinity_propagation(s_data, connections, iterations_number, s_s, t_p)
print("Сработало")
clusters = sorted(list(set(result.values())))
print("Количество кластеров:", len(clusters))

Итерация #1
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
21.85358500480652
Итерация #2
[0 0 0 ... 0 0 0]
[ 0  0  0 ... -3 -3 -3]
21.890467643737793
Итерация #3
[0 0 0 ... 0 0 0]
[-3 -3 -3 ... -3  0  0]
21.044742822647095
Итерация #4
[0 0 0 ... 0 0 0]
[-42 -42 -42 ...  -3  -3  -3]
21.068674564361572
Итерация #5
[0 0 0 ... 0 0 0]
[-12 -12 -12 ...  -3   0   0]
23.66075348854065
Итерация #6
[0 0 0 ... 0 0 0]
[-33 -33 -33 ...  -3  -3   0]
21.11953592300415
Итерация #7
[0 0 0 ... 0 0 0]
[-18 -18 -18 ...  -3   0   0]
24.048709630966187
Итерация #8
[0 0 0 ... 0 0 0]
[-30 -30 -30 ...  -3  -3   0]
21.275128602981567
Итерация #9
[0 0 0 ... 0 0 0]
[-18 -18 -18 ...  -3   0   0]
21.179352521896362
Итерация #10
[0 0 0 ... 0 0 0]
[-30 -30 -30 ...  -3  -3   0]
20.628878116607666
Сработало
Количество кластеров: 24801


In [7]:
clusters=(list(result.values()))
cluster_sizes = {}
for i in result.values():
    if i not in cluster_sizes:
        cluster_sizes[i] = 1
    else:
        cluster_sizes[i] += 1
print("Сработало")

Сработало


In [8]:
checkins_data = pd.read_csv("Gowalla_totalCheckins.txt", sep="	", header=None)[[0, 4]]
checkins_data.columns = ["user_id", "location_id"]

temp = sorted(list(result.values()))

clusters_data = pd.DataFrame(temp)
clusters_data = clusters_data.reset_index()
clusters_data.columns = ["user_id", "cluster_id"]

all_data = pd.merge(checkins_data, clusters_data, on=['user_id'])
valuable_users = np.unique(checkins_data.user_id)

train_data, test_data = train_test_split(valuable_users, test_size=0.1, shuffle=True)
train_checkins = all_data.loc[all_data.user_id.isin(train_data)]
test_checkins = all_data.loc[all_data.user_id.isin(test_data)]
top_locations = list(train_checkins.groupby(by = 'location_id')['location_id'].count().sort_values(ascending = False).iloc[:10].index)

temp = train_checkins.groupby(by = ["cluster_id", "location_id"])["location_id"].count()
train_ratings = temp.reset_index(name='rating').sort_values(by = ["cluster_id", "rating"], ascending = False)

print("Сработало")

Сработало


In [10]:
tp = 0
tp_cluster = 0

for i in test_data:
    cluster = clusters[i]
    locations = test_checkins.loc[test_checkins.user_id == i, "location_id"].values
    
    if (cluster in train_ratings.cluster_id):     
        top_cluster = train_ratings.loc[train_ratings.cluster_id == cluster, "location_id"].values        
        tp_cluster += get_tp(top_cluster, locations, 10)

    tp += get_tp(top_locations, locations, 10)

precision = tp/(len(test_data)*10)
cluster_precision = tp_cluster/(len(test_data)*10)

print(precision)
print(cluster_precision)

0.016974789915966387
0.0030812324929971988
